In [17]:
!pip install pandas

!pip install BeautifulSoup4
!pip install selenium
!pip install selenium --upgrade
!pip install webdriver-manager

In [76]:
import os
from datetime import datetime
import time
import requests
import re

import pandas as pd

from bs4 import BeautifulSoup

import selenium


from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select


In [192]:
# headless 모드를 위해 아래 3줄의 주석을 풀어주세요.

# option = webdriver.ChromeOptions()
# option.add_argument("--headless")

service = Service(executable_path='chromedriver.exe')
browser = webdriver.Chrome(service=service) #, options=option)

url = r"https://www.catch.co.kr/"

browser.maximize_window()
print(browser.get_window_size())

browser.get(url)

time.sleep(3)

{'width': 1552, 'height': 832}


In [193]:
from accounts import ct

ID = ct.ID
PW = ct.PW

In [194]:
# 로그인

browser.find_element(By.CSS_SELECTOR, "a.login").click()

browser.find_element(By.CSS_SELECTOR, "input#id_login").send_keys(ID)
time.sleep(1)
browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(PW)

browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(Keys.ENTER)

In [204]:
# 기업 검색 (input)
query = browser.find_element(By.CSS_SELECTOR, "input")

query.send_keys(input("리뷰 확인하기: "))
query.send_keys(Keys.ENTER)

In [205]:
### 팝업 뜨면 끄기 (예외 처리)

try : 
    browser.find_element(By.XPATH, r'//*[@id="layerInterest4"]/div/button[2]/span').click()
    time.sleep(2)

except : 
    pass

In [69]:
### 리뷰가 없어도 예외 처리

In [ ]:
#commentList > div.top > div > 

In [206]:
# 검색한 기업(최상위 노출) 클릭 
browser.find_element(By.CSS_SELECTOR, "p.name > a").click()
time.sleep(2)

In [221]:
# '현직자 리뷰' 탭 클릭
browser.find_element(By.XPATH, r'//*[@id="Contents"]/div[1]/div[1]/div[2]/ul/li[4]/a').click()
time.sleep(1)

In [225]:
# 핵심 리뷰만 조회 - 정규직/현직자가 3년 이내 작성한 리뷰
browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > div > a').click()
time.sleep(1)

In [211]:
# 긍정 리뷰만 조회
good_rv = browser.find_element(By.CSS_SELECTOR, "li:nth-child(1) > p:nth-child(4)").text
good_rv = good_rv.replace(good_rv[:5], "").replace('\n', ' ') 
good_rv

'정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준다. 짬순으로 정치순으로 진급 가능'

In [212]:
# 부정 리뷰만 조회
bad_rv = browser.find_element(By.CSS_SELECTOR, "li:nth-child(1) > p:nth-child(5)").text
bad_rv = bad_rv.replace(bad_rv[:4], "").replace('\n', ' ') 
bad_rv

'정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사원들의 업무를 독박으로 받아야 한다. 고여 있어서 최신 기술을 배울 수 없고 재택 근무가 불가능함'

In [213]:
good_df = pd.DataFrame({"긍정적 평가" : [good_rv]})
good_df

,긍정적 평가
0,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...


In [214]:
bad_df = pd.DataFrame({"부정적 평가" : [bad_rv]})
bad_df

,부정적 평가
0,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...


In [215]:
good_df.join(bad_df)

,긍정적 평가,부정적 평가
0,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...


In [276]:
# Web Crawling

n_btn = browser.find_element(By.CSS_SELECTOR, 'a.ico.next')

i = 0 

good_lst = []
bad_lst = []


while i < 123 : 


    for i in range(1, 5) : 

        grv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(4)").text
        brv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(5)").text

        grv_clean = grv.replace(grv[:5], "").replace('\n', ' ') 
        brv_clean = brv.replace(brv[:4], "").replace('\n', ' ') 

        good_lst.append(grv_clean)
        bad_lst.append(brv_clean)


    n_btn.click()
    

    time.sleep(1)

    i += 1

    if i == 100 :
        break

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"li:nth-child(4) > p:nth-child(5)"}
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00ECA813+48355]
	(No symbol) [0x00E5C4B1]
	(No symbol) [0x00D65358]
	(No symbol) [0x00D909A5]
	(No symbol) [0x00D90B3B]
	(No symbol) [0x00DBE232]
	(No symbol) [0x00DAA784]
	(No symbol) [0x00DBC922]
	(No symbol) [0x00DAA536]
	(No symbol) [0x00D882DC]
	(No symbol) [0x00D893DD]
	GetHandleVerifier [0x0112AABD+2539405]
	GetHandleVerifier [0x0116A78F+2800735]
	GetHandleVerifier [0x0116456C+2775612]
	GetHandleVerifier [0x00F551E0+616112]
	(No symbol) [0x00E65F8C]
	(No symbol) [0x00E62328]
	(No symbol) [0x00E6240B]
	(No symbol) [0x00E54FF7]
	BaseThreadInitThunk [0x759F7D59+25]
	RtlInitializeExceptionChain [0x76F2B74B+107]

In [277]:
len(good_lst), len(bad_lst)

(131, 131)

In [279]:
good_lst

['눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등',
 '고용안정성이 좋다. 사람이 많아 업무 진행이 수월하다.',
 '어디가서 이회사 다녀요 말할만함 부모님이 좋아하심 완전 개인근무',
 '눈치 볼 필요없이 칼퇴가 가능하다, 보너스를 많이 주는 편이다.',
 '밥 3끼, 출퇴근 버스가 생각보다 큰 장점. 교육 자기계발 지원이 많음. 대형 칩메이커라 배울 것이 많음',
 '복지 교육등 직원들의 성장에 많은 노력을 하고 부바부긴 하지만 눈치 안보고 휴가 쓰는게 가능하다.',
 '다니면 다닐 수록 대기업이라는 타이틀만 좋은 느낌이 드는 곳',
 '출퇴근 시간을 내 스케쥴 대로 할 수 있다. 눈치 안보고 휴가 쓸 수 있다.',
 '직원들 체력 증진을 위해 헬스장이 잘 되어있으나 수영장은 대기가 필요하다',
 '극강의 안정성이 최고입니다. HR 보장이 확고하여 휴가등이 보장됩니다',
 '눈치 없이 퇴근이 가능하나 부서바이부서가 심하다. 월급 만족',
 '식사를 제공하고, 자기계발비를 지원하지는 않지만 복지로 전자책/온라인 강의 수강권 등의 혜택으로 나오는 지원이 매달 있습니다. 부바부 팀바팀이겠지만 워라밸 및 근무환경이 좋아서 만족하고 다닙니다.',
 '네임벨류와 경기권 제조업 근무. 자율출퇴근제가 있다. 삼시세끼 무료 PI, PS가 있다.',
 '눈치 없이 퇴근할 수 있어서 좋다. 업무가 재밌고 끊임없이 도전할 수 있다',
 '밥값 들지 않음(밥 잘나옴), 교통비가 들지 않음(셔틀이 잘 되어 있어서)/ 커리어를 바꿀 기회가 있기는 함(원한다고 원하는 부서 갈 수 있는건 아님).',
 '셔틀이 많이 있다. 밥이 먹을만 하다 자질구레 하게 챙겨줌',
 '완벽한 근무환경 통합버스 운영으로 시간절약 높은 만족도로 일하고 있으며 자기개발 환경조성으로 커리어에 많은도움이 된다.',
 '워라밸, 사전에 계획하고 업무를 조정하면 휴가도 자유롭게 사용할 수 있는 점, 무엇보다 식당의 다양한 메뉴가 훌륭한 것 같다.',
 '근무지역이 좋다, 성과급이나 회사를 다니며

In [278]:
bad_lst

['위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성',
 '부바부가 심하고 경력전환의 기회가 적다. 최근에는 조직문화와 관련한 갈등이 많다',
 '대기업 치고 돈 조금준다 ㅠㅠ 이거 말곤 딱히 없는듯 경영진이 맘에 안듦',
 '서울에서 출퇴근시간이 오래 걸린다. 바쁠 땐 엄청 바쁘다,',
 '자유롭지 못한 조직문화. 보고를 위한 보고. 사내정치 매우 강함',
 '이 회사는 뭘해도 기준이 되기에 주변 눈치를 많이 보는듯 하고 최근들어 더욱 인건비를 줄이려고 하는듯함',
 '부바부 심하며 직원 복지가 조금씩 줄어 들고 있다는 느낌',
 '일이 너무 많다. 항상 일이 많다. 가끔은 일이 나를 절벽으로 몰아가고 있다는 생각이 들 때가 많다.',
 '경영진의 보여주기식 정치 보여주기식 복지가 너무 많음.',
 '사실 공무원 마인드가 있습니다. 망하지 않을 회사이기 때문입니다.',
 '빡센 부서는 야근을 밥 먹듯이 하고 꼰대 문화가 있다..',
 '부바부 팀바팀이 많이 큰 것 같습니다. 나쁜 점이라기보다는 회사가 커서 어쩔 수 없는 부분이라 생각합니다.',
 '부서 잘못 만나면 정시출근 야근이다. 연차 15개에 외에 창립기념일에도 안 쉼. 22년부터 재충전휴가 3개 생김 기본급은 낮고 상여 비율이 높아 PS안나오면 망함.',
 '부서바이부서가 매우 심하다. 업무가 많은 부서는 워라벨이 없다',
 '문화가 이상함. 자발적 노예가 많고, 일 안하는 것도 아니고 쫌 만 편해 보이면 동료도 아닌 사람들이 뒤에서 험담함. 보안 정책이 빡쎄서 딴 회사에서는 금방 할 일 보안정책이 촉매재가 되어 한없이 업무량이 늘어난다. 그리고 사람 아무리 많이 뽑아도 항상 부족한 느낌. 엘리트도 많지만 그렇지 않은 사람도 많다는 면에서, 엄청 실력이 뛰어 나다면 비추천. 사내게시판에서 이상한 글 많이 올라와서 볼 때 마다 내가 왜 이회사를 택했지라는 생각이 들게 만들어 줌. 성과급 안터지면 다닐 이유가 없는 회사인데 성과급이 2023년 이후로 잘 안터질 분위기이므로, 가성비 폭망.',


In [ ]:
# 2023-07-04 20:40 commit